<a href="https://colab.research.google.com/github/blancavazquez/diplomado_IA/blob/main/notebooks/Tema_01/03_regresion_lineal_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 11: Introduction to Neural Networks
# Topic 1: Neural Networks
## Regression: Linear & Logistc

## Phabel Antonio López Delgado



In [ ]:
# Install Libraries
!pip install statsmodels

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# StatsModel & Sklearn
import statsmodels.api as sm
from sklearn import linear_model

# Import Libraries for Kaggle API loading
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Plotly
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Random Seed
np.random.seed(8)

In [3]:
# Load Data with the KaggleDatasetAdapter
data_gymmembers = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "valakhorasani/gym-members-exercise-dataset",
    "gym_members_exercise_tracking.csv",
)
data_gymmembers.head()

100%|██████████| 63.6k/63.6k [00:00<00:00, 857kB/s]


,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


In [4]:
# Load Data with the KaggleDatasetAdapter
data_gymexercises = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "niharika41298/gym-exercise-data",
    "megaGymDataset.csv",
)
data_gymexercises.head()

100%|██████████| 657k/657k [00:00<00:00, 1.91MB/s]


,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [5]:
# Load Data with the KaggleDatasetAdapter
data_diseases = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "uom190346a/disease-symptoms-and-patient-profile-dataset",
    "Disease_symptom_and_patient_profile_dataset.csv",
)
data_diseases.head()

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Exercise 1: Linear Regresion

### Part I: Data Loading

In [6]:
# Load Data with the KaggleDatasetAdapter
data_gymmembers = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "valakhorasani/gym-members-exercise-dataset",
    "gym_members_exercise_tracking.csv",
)
data_gymmembers.describe()

,Age,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
count,973.000000,973.000000,973.00000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000
mean,38.683453,73.854676,1.72258,179.883864,143.766701,62.223022,1.256423,905.422405,24.976773,2.626619,3.321686,1.809866,24.912127
std,12.180928,21.207500,0.12772,11.525686,14.345101,7.327060,0.343033,272.641516,6.259419,0.600172,0.913047,0.739693,6.660879
min,18.000000,40.000000,1.50000,160.000000,120.000000,50.000000,0.500000,303.000000,10.000000,1.500000,2.000000,1.000000,12.320000
25%,28.000000,58.100000,1.62000,170.000000,131.000000,56.000000,1.040000,720.000000,21.300000,2.200000,3.000000,1.000000,20.110000
50%,40.000000,70.000000,1.71000,180.000000,143.000000,62.000000,1.260000,893.000000,26.200000,2.600000,3.000000,2.000000,24.160000
75%,49.000000,86.000000,1.80000,190.000000,156.000000,68.000000,1.460000,1076.000000,29.300000,3.100000,4.000000,2.000000,28.560000
max,59.000000,129.900000,2.00000,199.000000,169.000000,74.000000,2.000000,1783.000000,35.000000,3.700000,5.000000,3.000000,49.840000


## Part II: Plotly Automatic OLS Regression plot

In [7]:
# Automatic Regression with Ordinary Least Squares method with Plotly
fig = px.scatter(
    # Dataframe
    data_gymmembers,
    # Var x
    x='Session_Duration (hours)',
    # Var y
    y='Calories_Burned',
    # Opacity - alpha
    opacity=0.65,
    # Method: Ordinary Least Squares
    trendline='ols',
    # Regression Line Color
    trendline_color_override='red'
)

# Plot
fig.show()

### Part III: StatsModels Method


In [8]:
# Split Data
x = data_gymmembers['Session_Duration (hours)']
y = data_gymmembers['Calories_Burned']

# Model
x_data = sm.add_constant(x)

# Fit
modelo_stat = sm.OLS(y, x).fit()

# Model Summary
modelo_stat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:        Calories_Burned   R-squared (uncentered):                   0.985
Model:                            OLS   Adj. R-squared (uncentered):              0.985
Method:                 Least Squares   F-statistic:                          6.579e+04
Date:                Tue, 24 Jun 2025   Prob (F-statistic):                        0.00
Time:                        01:52:27   Log-Likelihood:                         -5989.7
No. Observations:                 973   AIC:                                  1.198e+04
Df Residuals:                     972   BIC:                                  1.199e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Session_Duration (hours)   720.7146      2.810    256.502      0.000     715.201     726.228
==============================================================================
Omnibus:                        9.896   Durbin-Watson:                   2.086
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               10.033
Skew:                           0.218   Prob(JB):                      0.00663
Kurtosis:                       3.238   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
# Graph StatsModel Regression Plot
fig = px.scatter(
    # Dataframe
    data_gymmembers,
    # Var x
    x='Session_Duration (hours)',
    # Var y
    y='Calories_Burned',
    # Opacity - alpha
    opacity=0.65,
    # Method
    #trendline='ols',
    # Regression Line Color
    #trendline_color_override='darkblue'
)

# Add StatsModel Regression Fit
yhat = 720.7146*x + 2.810
fig.add_traces(go.Scatter(x=x, y=yhat, name='Regression Fit'))

# Plot
fig.show()

## Exercise II: Logistic Regression

### Part I: Data Loading

In [10]:
# Load Data with the KaggleDatasetAdapter
data_gymmembers = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "valakhorasani/gym-members-exercise-dataset",
    "gym_members_exercise_tracking.csv",
)
data_gymmembers.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


### Part II: Data Ploting

In [11]:
# Automatic Regression with Ordinary Least Squares method with Plotly
fig = px.scatter(
    # Dataframe
    data_gymmembers,
    # Var x
    x='Session_Duration (hours)',
    # Var y
    y='Calories_Burned',
    # Color
    color='Gender',
    # Opacity - alpha
    opacity=0.65,
    # Method
    trendline='ols',
    # Regression Line Color
    #trendline_color_override=['red', 'blue']
)

# Plot
fig.show()

### Part III: Regresion Sklearn Model

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [13]:
# Set Data
x = data_gymmembers[['Session_Duration (hours)', 'Calories_Burned', 'Fat_Percentage']]
y = data_gymmembers['Gender']

# Split Data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [14]:
# Model
clf = linear_model.LogisticRegression(C=1e40, solver='newton-cg')

# Fit
modelo = clf.fit(X_train, y_train)

# Results
print("Theta_0: ", modelo.intercept_ )
print("For theta_1 and theta_2: ", modelo.coef_ )

Theta_0:  [9.96368455]
For theta_1 and theta_2:  [[-8.47799751e+00  8.10227736e-03 -2.59458548e-01]]


In [18]:
# Predict Sex, given 3 vars:['Session_Duration (hours)', 'Calories_Burned', 'Fat_Percentage']
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Female       0.79      0.78      0.78        86
        Male       0.83      0.83      0.83       109

    accuracy                           0.81       195
   macro avg       0.81      0.81      0.81       195
weighted avg       0.81      0.81      0.81       195



## Ejercicio 3: MultiClass Classification

### Part I: Data Loading

In [60]:
# Load Data with the KaggleDatasetAdapter
data_gymmembers = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "valakhorasani/gym-members-exercise-dataset",
    "gym_members_exercise_tracking.csv",
)
data_gymmembers.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


### Part II: Correlation Analysis

In [62]:
# Encode Vars
from sklearn.preprocessing import LabelEncoder

# Encode Workout Type
le_workout = LabelEncoder()
le_workout.fit(data_gymmembers['Workout_Type'].unique())
data_gymmembers['Workout_Type'] = le_workout.transform(data_gymmembers['Workout_Type'])

# Encode Gender
le_sex = LabelEncoder()
le_sex.fit(data_gymmembers['Gender'].unique())
data_gymmembers['Gender'] = le_sex.transform(data_gymmembers['Gender'])

In [65]:
# Look for Correlation
data_corr = data_gymmembers.corr().round(2)

# Plotly
fig = px.imshow(data_corr, text_auto=True, color_continuous_scale="magma", title="Correlation Heatmap")
fig.update_layout(width=1200, height=1000)
fig.show()

In [71]:
# Reduced Corr Plot
from math import nan
data_corr[abs(data_corr) < 0.6] = nan

# Plotly -> "Workout_Type" has not noticeable correlation with any other var
fig = px.imshow(data_corr, text_auto=True, color_continuous_scale="magma", title="Correlation Heatmap")
fig.update_layout(width=1200, height=1000)
fig.show()

### Part III: Multiclass Logistic Regression Model

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [67]:
# Set Data
x = data_gymmembers[['Session_Duration (hours)', 'Calories_Burned', 'Workout_Frequency (days/week)', 'Fat_Percentage']]
y = data_gymmembers['Experience_Level']

# Split Data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [68]:
# Cross-Validation Function
from sklearn.linear_model import LogisticRegressionCV

# Set Model
clf = linear_model.LogisticRegressionCV(Cs=10, solver='newton-cg', cv=10)

# Fit
model = clf.fit(X_train, y_train)

# Results
print("Theta_0: ", model.intercept_ )
print("For theta_1 - theta_n: ", model.coef_ )

Theta_0:  [ -0.10412633 -21.01531375  21.11944008]
For theta_1 - theta_n:  [[-1.79181584e+00 -6.29650813e-03 -3.97835176e+00  1.03948327e+00]
 [ 2.67789666e-01 -3.38703526e-03  1.51151584e+00  1.03031475e+00]
 [ 1.52402617e+00  9.68353908e-03  2.46683591e+00 -2.06979802e+00]]


In [69]:
# Predict Workout Type, given n vars: ['Session_Duration (hours)', 'Calories_Burned', 'Workout_Frequency (days/week)', 'Fat_Percentage']
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.84      0.82      0.83        77
           2       0.82      0.85      0.84        78
           3       1.00      1.00      1.00        40

    accuracy                           0.87       195
   macro avg       0.89      0.89      0.89       195
weighted avg       0.87      0.87      0.87       195

